<a href="https://colab.research.google.com/github/DmitryiLaptev/NeuralNetworks_HW/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_3%2C_%D0%94_%D0%97_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_(%D0%9B%D0%B0%D0%BF%D1%82%D0%B5%D0%B2_%D0%94_%D0%90_)_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задача 3. Автокодировщик** (10 баллов) 

Провести анализ модели автокодировщика (не вариационного) для [выборки Twitter](https://drive.google.com/drive/folders/1NdhIpkVTDBDyal-KM54XI2GvqOtFbJlD?usp=sharing) (эмбединги предложений). Требуется сравнить качество востановления предложения в зависимости от размера слоя, числа слоев, dropout, BatchNorm, размера словаря. Все выводы должны быть представленны в формате TensorBoard.


In [ ]:
# Загрузка библиотек
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec
from PIL import Image
import io
import os
from urllib.request import urlopen
from skimage.segmentation import mark_boundaries

from tqdm.notebook import tqdm
import numpy as np
import requests
from scipy.stats import norm
import torch

from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import random_split, DataLoader

from torchvision import datasets, transforms
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Установка AutoTokenizer и torchmetrics
! pip install --quiet transformers
! pip install --quiet torchmetrics

from transformers import AutoTokenizer
import torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 13.9 MB/s eta 0:00:00


In [ ]:
# Определим устройство на котором происходят вычисления
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
# Выгрузка данных
import pandas as pd

dataset = pd.read_csv('/content/twitter.csv')
dataset = dataset.sample(10000, random_state=42)
dataset.sample(10, random_state=42)

,tag,message
204869,0.0,@gabsramazzina brigadeiro? what are you trying...
1406138,0.0,"Get the bill of being lazy, and I have no mone..."
1404132,0.0,Forgot i bought a packet of monster munch earl...
751245,1.0,"Girls to do the dishes, girls to clean up my r..."
298055,1.0,@MariKurisato now is back working...what she d...
357045,0.0,@notoastmaker baked mac and cheese it was REA...
278687,1.0,@joeymcintyre &quot;suck the nectar&quot; -- t...
1538277,0.0,"still bored.... very, very, i mean really BORE..."
437885,1.0,@sky327 Thanks! I'm a bit hoarse from screaming
1364350,1.0,23 days until i'm 10 rows away from dane cook....


In [ ]:
train_mask = np.random.rand(len(dataset), ) < 0.8
train_ds = dataset[train_mask]
val_ds = dataset[~train_mask]

train = train_ds['message'].to_list()
val   = val_ds['message'].to_list()

In [ ]:
# Токенизатор
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE", verbose=False)

tokenized_train_codes = tokenizer(train, return_tensors='pt', max_length=64, padding=True, truncation=True)['input_ids']
tokenized_val_codes   = tokenizer(val,   return_tensors='pt', max_length=64, padding=True, truncation=True)['input_ids']

print(f'init size of vocab: {tokenizer.vocab_size}')

init size of vocab: 501153


In [ ]:
# Уменьшим размер словаря
new_vocab = {tokenizer.pad_token_id:0, tokenizer.unk_token_id:1}
for token in tokenized_train_codes.unique().tolist():
  if token not in new_vocab:
    new_vocab[token] = len(new_vocab.values())

print(f'reduced size of vocab: {len(new_vocab)}')

reduced size of vocab: 17563


In [ ]:
# Функция tokenize
def tokenize(tokenized_data_in_num, new_vocab):
  new_tokenized_data = []
  for sent in tokenized_data_in_num:
    new_tokenized_sentence = []
    for num in sent:  
      num = int(num)
      if num in new_vocab :
        new_tokenized_sentence.append(int(new_vocab[num]))
      else:
        new_tokenized_sentence.append(int(new_vocab[tokenizer.unk_token_id]))
    new_tokenized_data.append(new_tokenized_sentence)

  return new_tokenized_data

In [ ]:
tokenized_train = tokenize(tokenized_train_codes, new_vocab)
tokenized_val   = tokenize(tokenized_val_codes,   new_vocab)

train_ds = torch.utils.data.TensorDataset(torch.tensor(tokenized_train), torch.tensor(tokenized_train))
val_ds   = torch.utils.data.TensorDataset(torch.tensor(tokenized_val), torch.tensor(tokenized_val))

BATCH_SIZE = 32

train_dl = torch.utils.data.DataLoader(train_ds, batch_size = BATCH_SIZE)
val_dl   = torch.utils.data.DataLoader(val_ds, batch_size = BATCH_SIZE)

In [ ]:
# Функция энкодера
def encode_sentence(sentence, tokenizer, new_vocab):
  tokenized_sentence = tokenizer.encode(sentence)
  new_tokenized_sentence = []
  for num in tokenized_sentence:  
      num = int(num)
      if num in new_vocab:
        new_tokenized_sentence.append(new_vocab[num])
      else:
        new_tokenized_sentence.append(new_vocab[tokenizer.unk_token_id])
  
  return new_tokenized_sentence

In [ ]:
# Функция декодера
def decode_sentence(tokenized_sentence, tokenizer, new_vocab) :
  vocab = dict(map(lambda pair: (pair[1], pair[0]), new_vocab.items()))
  old_tokenized_sentence = []
  for token in tokenized_sentence:
    old_tokenized_sentence.append(vocab[token])

  sentence = tokenizer.decode(old_tokenized_sentence)

  return sentence

In [ ]:
# Создание модели 
class Encoder(torch.nn.Module):
  def __init__(self, vocab_size, emb_dim=30, hidden_dim=30, p=0):
    super(Encoder, self).__init__()
    self.embedding = torch.nn.Embedding(vocab_size, emb_dim)
    self.lstm = torch.nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True, dropout=p)

  def forward(self, input):
    input = self.embedding(input)
    out, h_c = self.lstm(input)
    return out, h_c

In [ ]:
# Создание модели 
class Decoder(torch.nn.Module):
  def __init__(self, vocab_size, encoded_dim, hidden_dim=30, p=0):
    super(Decoder, self).__init__()
    self.lstm = torch.nn.LSTM(encoded_dim, hidden_dim, batch_first=True, bidirectional=True, dropout=p)
    self.linear = torch.nn.Linear(2*hidden_dim, vocab_size)
    self.dropout = torch.nn.Dropout(p)

  def forward(self, encoder_output):
    x, h_c = encoder_output
    out, h_c = self.lstm(x, h_c)
    out = self.dropout(self.linear(out))
    return out

In [ ]:
# Создание модели 
class AutoEncoder(torch.nn.Module):
  def __init__(self, vocab_size, latent_dim=30, emb_dim=30, hidden_dim=30, 
               num_layers=1, p=0, zero_input=True):
    super(AutoEncoder, self).__init__()

    self.num_classes = vocab_size
    
    self.encoder = Encoder(vocab_size, emb_dim, hidden_dim, p)
    self.decoder = Decoder(vocab_size, 2*hidden_dim, hidden_dim, p)

    self.hidden_dim = hidden_dim
    self.latent_dim = latent_dim
    self.emb_dim = emb_dim
    self.num_layers = num_layers
    self.dropout = torch.nn.Dropout(p)
    self.p = p

  def forward(self, input):
    encoder_out = self.encoder(input)
    decoder_out = self.decoder(encoder_out) 
    out = decoder_out.transpose(1, 2)
    return out

In [ ]:
# Вычисление точности загрузки данных
def calc_accuracy_dl(model, dl, writer, name_dl=None, num_step=0) :
  sum_acc = 0
  for i, (features, labels) in enumerate(dl):
    if i > (BATCH_SIZE * 2) :
      break
    features = features.to(device)
    labels = labels.to(device)
    preds = model(features)

    sum_acc += torchmetrics.functional.accuracy(preds, labels, task='multiclass', num_classes=model.num_classes, ignore_index=0)

  acc = sum_acc / (BATCH_SIZE * 2)
  
  writer.add_scalars(main_tag = f'{name_dl} accuracy', 
                     tag_scalar_dict={f'vocab_size = {model.num_classes}, latent_dim={model.latent_dim}, emb_dim={model.emb_dim}, hidden_dim={model.hidden_dim}, num_layers={model.num_layers}, drop_p={model.p},': acc}, 
                     global_step=num_step)
  return acc

def calc_all_accuracy(model, num_epoch, epochs, train_dl, val_dl, writer, num_step, dump_p=False) :
  train_acc = calc_accuracy_dl(model, train_dl, writer, name_dl='train', num_step=num_step)
  val_acc   = calc_accuracy_dl(model, val_dl,   writer, name_dl='valid', num_step=num_step)
  if dump_p:
    print(f"[{num_epoch}/{epochs}]: train accuracy = {train_acc}, validation accuracy = {val_acc}")
  return val_acc, train_acc

def dump_to_tensorboard(model, num_epoch, val_dl, writer) :
  preds = None
  labels = None
  for i, (features, labels) in enumerate(val_dl):
    features = features.to(device)
    labels = labels.to(device)
    preds = model(features)
    break

  preds = torch.argmax(preds, 1)[0].cpu().tolist()
  targets = labels[0].cpu().tolist()

  decoded_prediction = decode_sentence(preds, tokenizer, new_vocab)

  writer.add_text(tag=f'val_preds: vocab_size = {model.num_classes}, latent_dim={model.latent_dim}, emb_dim={model.emb_dim}, hidden_dim={model.hidden_dim}, num_layers={model.num_layers}, drop_p={model.p}',
                  text_string=decoded_prediction, 
                  global_step=num_epoch)

In [ ]:
# Обучение
def train(model, optimizer, train_dl, val_dl, writer, epochs=30, device=device):
  model.to(device)

  num_step = 0
  
  calc_all_accuracy(model, 0, epochs, train_dl, val_dl, writer, num_step, dump_p=True)

  best_model = model
  best_acc = 0
  for epoch in range(0, epochs):
    model.train()
    for i, (features, labels) in enumerate(train_dl): 
      
      num_step = num_step + 1
      if (num_step) % (BATCH_SIZE * 2) == 0:
          calc_all_accuracy(model, epoch + 1, epochs, train_dl, val_dl, writer, num_step, dump_p=False)

      features, labels = features.to(device), labels.to(device)
      optimizer.zero_grad()
      
      pred = model(features)
      loss = torch.nn.functional.cross_entropy(pred, labels, ignore_index=0)
      loss.backward()
      
      optimizer.step()

    val_acc, _ = calc_all_accuracy(model, epoch + 1, epochs, train_dl, val_dl, writer, num_step, dump_p=True)
    dump_to_tensorboard(model, epoch + 1, val_dl, writer)

    if (val_acc > best_acc):
      best_model = deepcopy(model)
      best_acc = val_acc
  
  return best_model

In [ ]:
! rm -rf './runs'

In [ ]:
writer = SummaryWriter()

num_layerss = [2, 4]
hidden_dims = [200, 400]
drop_ps = [0.1, 0.3]

for num_layers in num_layerss :
  for hidden_dim in hidden_dims :
    for drop_p in drop_ps:
      print (f'\n num_layers = {num_layers}, hidden_dim = {hidden_dim}, drop_p = {drop_p}')
      model = AutoEncoder(vocab_size=len(new_vocab), latent_dim=30, emb_dim=30, hidden_dim=hidden_dim, 
                          num_layers=num_layers, p=drop_p, zero_input=True)
      optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
      train(model, optimizer, train_dl, val_dl, writer, epochs=3, device=device)


 num_layers = 2, hidden_dim = 200, drop_p = 0.1


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


[0/3]: train accuracy = 2.2677793822367676e-05, validation accuracy = 0.0
[1/3]: train accuracy = 0.5653191804885864, validation accuracy = 0.5372605919837952
[2/3]: train accuracy = 0.738260805606842, validation accuracy = 0.6921279430389404
[3/3]: train accuracy = 0.807630181312561, validation accuracy = 0.7486963272094727

 num_layers = 2, hidden_dim = 200, drop_p = 0.3


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


[0/3]: train accuracy = 0.0001920332433655858, validation accuracy = 0.00017654320981819183
[1/3]: train accuracy = 0.5293636918067932, validation accuracy = 0.5029727816581726
[2/3]: train accuracy = 0.6249050498008728, validation accuracy = 0.5829392075538635
[3/3]: train accuracy = 0.6633298993110657, validation accuracy = 0.6109623312950134

 num_layers = 2, hidden_dim = 400, drop_p = 0.1
[0/3]: train accuracy = 0.0, validation accuracy = 0.0
[1/3]: train accuracy = 0.7279384732246399, validation accuracy = 0.6916937232017517
[2/3]: train accuracy = 0.834759533405304, validation accuracy = 0.775297999382019
[3/3]: train accuracy = 0.8833275437355042, validation accuracy = 0.8015109896659851

 num_layers = 2, hidden_dim = 400, drop_p = 0.3
[0/3]: train accuracy = 6.173881411086768e-05, validation accuracy = 2.3602719011250883e-05
[1/3]: train accuracy = 0.594819188117981, validation accuracy = 0.5644838809967041


In [ ]:
writer.close()

In [ ]:
# Выведем все данные в тензорбод
%load_ext tensorboard
%tensorboard --logdir ./runs